In [41]:
from torch.utils.data import DataLoader
import os
import sys
sys.path.append('..')
sys.path.append('../..')
from utils.Network import Network
from utils.Analyser import Analyser
from utils.io import save_network, load_network, save, load, create_results_folder
from utils.WaveDataset import create_datasets, transformVar, normalize
from utils.arg_extract import get_args
from utils.Scorekeeper import Scorekeeper

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from collections import OrderedDict
from utils.arg_extract import get_args
from utils.ExperimentBuilder import ExperimentBuilder
from utils.Model_ConvLSTM import EncoderForecaster, Encoder, Forecaster, ConvLSTMCell

In [42]:
num_workers=1

In [43]:
batch_size=16

In [44]:
base_folder = '/Users/stathis/Code/thesis/wave_propagation'

test_dataset, val_dataset, train_dataset = create_datasets(os.path.join(base_folder,"Video_Data/"), 
                                                           transformVar, 
                                                            test_fraction=0.15, validation_fraction=0.15)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

In [45]:
inputs = torch.rand(16,1,128,128)
# mod421 = nn.Conv2d(1,1,4,2,1)
mod321 = nn.Conv2d(1,1,3,2,1)
up321 = nn.ConvTranspose2d(1,1,3,2,1)
up421 = nn.ConvTranspose2d(1,1,4,2,1)
print(inputs.shape)
out = mod321(inputs)
print(out.shape)
out = mod321(out)
print(out.shape)
out = mod321(out)
print(out.shape)
out = up421(out)
print(out.shape)
out = up421(out)
print(out.shape)
out = up421(out)
print(out.shape)

torch.Size([16, 1, 128, 128])
torch.Size([16, 1, 64, 64])
torch.Size([16, 1, 32, 32])
torch.Size([16, 1, 16, 16])
torch.Size([16, 1, 32, 32])
torch.Size([16, 1, 64, 64])
torch.Size([16, 1, 128, 128])


In [46]:
batch_size = 16
device='cpu'
seq_input=5
seq_output=20
seq_length=25

encoder_architecture = [
    [ #in_channels, out_channels, kernel_size, stride, padding
        OrderedDict({'conv1_leaky_1': [1, 8, 3, 2, 1]}),
        OrderedDict({'conv2_leaky_1': [64, 192, 3, 2, 1]}),
        OrderedDict({'conv3_leaky_1': [192, 192, 3, 2, 1]}),
    ],

    [
        ConvLSTMCell(input_channel=8, num_filter=64, b_h_w=(batch_size, 64, 64),
                 kernel_size=3, stride=1, padding=1,device=device,seq_len=seq_input),
        ConvLSTMCell(input_channel=192, num_filter=192, b_h_w=(batch_size, 32, 32),
                 kernel_size=3, stride=1, padding=1,device=device,seq_len=seq_input),
        ConvLSTMCell(input_channel=192, num_filter=192, b_h_w=(batch_size, 16, 16),
                 kernel_size=3, stride=1, padding=1,device=device,seq_len=seq_input),
    ]
]
forecaster_architecture = [
    [
        OrderedDict({'deconv1_leaky_1': [192, 192, 4, 2, 1]}),
        OrderedDict({'deconv2_leaky_1': [192, 64, 4, 2, 1]}),
        OrderedDict({
            'deconv3_leaky_1': [64, 8, 4, 2, 1],
            'conv3_leaky_2': [8, 8, 3, 1, 1],
            'conv3_3': [8, 1, 1, 1, 0]
        }),
    ],

    [
        ConvLSTMCell(input_channel=192, num_filter=192, b_h_w=(batch_size, 16, 16),
                 kernel_size=3, stride=1, padding=1,device=device,seq_len=seq_output),
        ConvLSTMCell(input_channel=192, num_filter=192, b_h_w=(batch_size, 32, 32),
                 kernel_size=3, stride=1, padding=1,device=device,seq_len=seq_output),
        ConvLSTMCell(input_channel=64, num_filter=64, b_h_w=(batch_size, 64, 64),
                 kernel_size=3, stride=1, padding=1,device=device,seq_len=seq_output),
    ]
]

encoder = Encoder(encoder_architecture[0],encoder_architecture[1]).to(device)
forecaster=Forecaster(forecaster_architecture[0],forecaster_architecture[1],
                      seq_output).to(device)
model = EncoderForecaster(encoder,forecaster)


In [47]:
# inputs = torch.rand(5,16,1,128,128)

In [48]:
batch = next(iter(train_dataloader))

In [49]:
batch.size()

torch.Size([16, 100, 128, 128])

In [50]:
from utils.helper_functions import convert_SBCHW_to_BSHW,convert_BSHW_to_SBCHW

In [51]:
out = convert_BSHW_to_SBCHW(batch)

In [52]:
output = model(batch)

0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [53]:
output.size()

torch.Size([16, 20, 128, 128])